In [1]:
import os

# Changing working directory to root to use custom libs
os.chdir('/opt/workspace/')

# Libs import
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import date_format, col, to_date, isnan
import pyspark
import sys
from pathlib import Path
sys.path.append('/opt/workspace/')
from pyspark.sql.utils import AnalysisException
from pathlib import Path
import yaml
from magaluTools.logger import get_logger
from magaluTools.dataValidation import *
from magaluTools.etl import src2raw


In [2]:
#Spark Configurations
    # Sets Session to use spark master container
    # Sets Session to use warehouse directory in /opt/workspace/Warehouse and to infer data Schema
    # Sets overide mode to dynamic, so we can append data and overwrite old data based on partition
spark = SparkSession.builder.appName('warehouse_setup') \
        .master("spark://spark-master:7077") \
        .config("spark.sql.streaming.schemaInference", True) \
        .config("spark.sql.warehouse.dir", '/opt/workspace/Warehouse') \
        .enableHiveSupport() \
        .config("spark.sql.sources.partitionOverwriteMode", 'dynamic').getOrCreate()

In [7]:
spark.sql('describe curated.partners_commissions_by_month').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|         id_parceiro|   string|   null|
|partner_commissio...|   double|   null|
|       partner_bonus|   bigint|   null|
|partner_commissio...|   double|   null|
|commission_month_...|   string|   null|
|# Partition Infor...|         |       |
|          # col_name|data_type|comment|
|commission_month_...|   string|   null|
+--------------------+---------+-------+



In [3]:
# Creating warehouse databases

spark.sql('create database if not exists raw')
spark.sql('create database if not exists curated')

DataFrame[]

In [4]:
# Getting the metadata for correctly dumping the csv files to the warehouse

with open('./dump_metadata.yml', 'r') as f:
            dump_metadata = yaml.load(f)

<ipython-input-4-7a215d74acdc>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dump_metadata = yaml.load(f)


In [5]:
# Iterating over the files in /Data/ Folder to dump the data to the Warehouse

for x in Path('./Data').iterdir():
    
    # Checking if the the current entity is a file or directory
    if not x.is_dir(): 
        
        # Seting the name of the table
        table_name = x.name.replace('.csv','')
        
        # Checking if the metadata needed for the etl is in the file, if not, skips the file
        if table_name in dump_metadata:
            
            # Getting the metadata specicallly for the current dump
            metadata = dump_metadata[table_name]
            
            # Using custom function to extract the file data and transform it into parquet files in the warehouse
            result_df = src2raw(file_path=f'/opt/workspace/Data/{x.name}',
                                table_name=table_name,
                                validations=metadata.get('validations'),
                                partition_data=metadata.get('partition'),
                                csv_delimiter=metadata.get('delimiter'),
                                raise_if_validation_failed=True)
    
        # Printing warning for the user to know the metadata for the csv file was not found
        else:
            get_logger().warning(f'CSV metadata not found for {x.name}, please create it in the dump_metadata.yml file :D')

[16-Sep-21 20:49:16 - MainProcess - MainThread] - INFO: [DATA VALIDATION] Unique Key Check for column id_categoria. RESULT: PASSED
[16-Sep-21 20:49:17 - MainProcess - MainThread] - INFO: [DATA VALIDATION] Nullity Check for column perc_parceiro. RESULT: PASSED
[16-Sep-21 20:49:23 - MainProcess - MainThread] - INFO: [DATA VALIDATION] Nullity Check for column id_estado. RESULT: PASSED
[16-Sep-21 20:49:23 - MainProcess - MainThread] - INFO: [DATA VALIDATION] Nullity Check for column id_cidade. RESULT: PASSED
[16-Sep-21 20:49:38 - MainProcess - MainThread] - INFO: [DATA VALIDATION] Unique Key Check for column id_cliente. RESULT: PASSED
[16-Sep-21 20:49:39 - MainProcess - MainThread] - INFO: [DATA VALIDATION] Nullity Check for column nm_cliente. RESULT: PASSED
[16-Sep-21 20:49:48 - MainProcess - MainThread] - INFO: [DATA VALIDATION] Unique Key Check for column id_filial. RESULT: PASSED
[16-Sep-21 20:49:49 - MainProcess - MainThread] - INFO: [DATA VALIDATION] Unique Key Check for column ds_fi

In [6]:
# Stops Spark and finishes the job
spark.stop()